In [ ]:
import MLJ
import DataFrames as DF
import CSV
using DataFramesMeta
using StatsPlots

In [ ]:
include("PreprocessAerosolData.jl")
include("ActivationEmulatorModels.jl")

In [ ]:
X_test, Y_test = read_aerosol_dataset("datasets/2modal_dataset2_test.csv")

In [ ]:
mach = MLJ.machine("emulators/2modal_nn_machine5.jls")

In [ ]:
pred_S_max = exp.(MLJ.predict(mach, X_test).log_S_max)
ARG_S_max = exp.(preprocess_aerosol_data(X_test).log_ARG_S_max)
PySDM_S_max = exp.(Y_test.log_S_max)

In [ ]:
minpt = min(minimum(PySDM_S_max), minimum(ARG_S_max), minimum(pred_S_max))
maxpt = max(maximum(PySDM_S_max), maximum(ARG_S_max), maximum(pred_S_max))

function plot_ref_lines!()
    plot!([minpt, maxpt], [minpt, maxpt], color=:red, label=nothing)
    plot!([minpt, maxpt], [minpt * 2, maxpt * 2], color=:blue, label=nothing)
    plot!([minpt, maxpt], [minpt / 2, maxpt / 2], color=:blue, label=nothing)
end

p1 = scatter(PySDM_S_max, ARG_S_max, color=:black, markersize=1,
            markerstrokewidth=0, label=nothing, xscale=:log10, yscale=:log10,
            title="ARG", xlabel="PySDM S_max", ylabel="Predicted S_max")
plot_ref_lines!()
p2 = scatter(PySDM_S_max, pred_S_max, color=:black, markersize=1,
            markerstrokewidth=0, label=nothing, xscale=:log10, yscale=:log10,
            title="Emulator", xlabel="PySDM S_max", ylabel="Predicted S_max")
plot_ref_lines!()
plot(p1, p2)
savefig("plots/model_comparison.pdf")

In [ ]:
p1 = histogram(ARG_S_max ./ PySDM_S_max, label=nothing, xlabel="predicted / true S_max", title="ARG")
p2 = histogram(pred_S_max ./ PySDM_S_max, label=nothing, xlabel="predicted / true S_max", title="Emulator")
plot(p1, p2)